## Graphs of incompleteness by protein, incompleteness by drug perturbation


In [47]:
import pandas as pd
BT549_data=pd.read_csv(r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Experiments\raw_data\TNBC_set\BT549_processed_data.tsv',sep='\t')
DU4475_data=pd.read_csv(r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Experiments\raw_data\TNBC_set\DU4475_processed_data.tsv',sep='\t')
HCC70_data=pd.read_csv(r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Experiments\raw_data\TNBC_set\HCC70_processed_data.tsv',sep='\t')
HS578T_data=pd.read_csv(r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Experiments\raw_data\TNBC_set\HS578T_processed_data.tsv',sep='\t')
MCF7_data=pd.read_csv(r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Experiments\raw_data\TNBC_set\MCF7_processed_data.tsv',sep='\t')
MDA_MB_453_data=pd.read_csv(r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Experiments\raw_data\TNBC_set\MDA-MB-453_processed_data.tsv',sep='\t')
viabilities_data=pd.read_csv(r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Experiments\raw_data\TNBC_set\cell_viability_sampleinfo_for_chris.csv')

# Drop rows that have BT549, DU4475, HCC70, HS578T, or MCF7 in any cell entry
cell_lines_to_exclude = ['BT549', 'DU4475', 'HCC70', 'HS578T', 'MCF7','BT20', 'HCC1143','HCC1395','HCC1806']

def row_contains_cell_line(row):
    return any(str(cell_line) in str(cell) for cell in row for cell_line in cell_lines_to_exclude)

# Apply the filter to all loaded dataframes

viabilities_data = viabilities_data[~viabilities_data.apply(row_contains_cell_line, axis=1)]
# Drop rows where 'pert_time' is 48 or 6
viabilities_data = viabilities_data[~viabilities_data['pert_time'].isin([48, 6])]
viabilities_data = viabilities_data[~viabilities_data['BioRep'].isin([2, 3])]

viabilities_data=viabilities_data.iloc[:, list(range(0, 7)) + [8]]
# Trim off the last '_24' from the column names in the MDA_MB_453_data dataframe
MDA_MB_453_data.columns = [col[:-3] if col.endswith('_24') else col for col in MDA_MB_453_data.columns]

# Drop columns in MDA_MB_453_data that end with '_48' or '_6'
cols_to_drop = [col for col in MDA_MB_453_data.columns if col.endswith('_48') or col.endswith('_6')]
MDA_MB_453_data = MDA_MB_453_data.drop(columns=cols_to_drop)

sample_info = pd.read_csv(r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Experiments\raw_data\TNBC_set\PTV1_sample_info_test.csv')

# Keep only the first instance of each pert_id in sample_info
sample_info = sample_info.drop_duplicates(subset='pert_id', keep='first')
sample_info=sample_info.iloc[:,:3]

# For each column in the MDA_MB_453_data dataframe, take the substring after the first underscore (if present)
# and append it to the front of the column name, separated by an underscore.
def prepend_after_underscore(col):
    if '_' in col:
        after_ = col.split('_', 1)[1]
        return f"{after_}_{col}"
    else:
        return col

MDA_MB_453_data.columns = [prepend_after_underscore(col) for col in MDA_MB_453_data.columns]

MDA_MB_453_data=MDA_MB_453_data.transpose()
MDA_MB_453_data.columns = MDA_MB_453_data.iloc[0]
MDA_MB_453_data['cell viability'] = None
MDA_MB_453_data = MDA_MB_453_data.iloc[1:]
# Remove '#' from the values in the 'pert_id' column in the viabilities_data DataFrame
if 'Pert_ID' in viabilities_data.columns:
    viabilities_data['Pert_ID'] = viabilities_data['Pert_ID'].astype(str).str.replace('#', '', regex=False)

for idx, row in viabilities_data.iterrows():
    pert_id = str(row['Pert_ID'])
    inhi_5_value = row['Inhi_5']
    # Find all indices in MDA_MB_453_data that contain the pert_id as a substring
    matching_indices = [i for i in MDA_MB_453_data.index if pert_id in str(i)]
    for match_idx in matching_indices:
        MDA_MB_453_data.at[match_idx, 'cell viability'] = inhi_5_value

MDA_MB_453_data['drug name'] = MDA_MB_453_data.index
MDA_MB_453_data = MDA_MB_453_data.reset_index(drop=True)
# Split the 'drug name' column into 'pert_number' and 'drug_name_clean'
MDA_MB_453_data['pert_number'] = MDA_MB_453_data['drug name'].apply(lambda x: str(x).split('_')[0] if '_' in str(x) else str(x))
MDA_MB_453_data['drug_name_clean'] = MDA_MB_453_data['drug name'].apply(
    lambda x: ''.join([c for c in str(x).split('_', 1)[-1] if not c.isdigit() and c != '_']) if '_' in str(x) else ''.join([c for c in str(x) if not c.isdigit() and c != '_'])
)
MDA_MB_453_data=MDA_MB_453_data.drop(columns=['drug name'])

drug_names_targets=pd.read_csv(r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Experiments\raw_data\TNBC_set\ptv1_unique_drug_names.csv')
MDA_MB_453_data['targeted_uniprots']=None



In [48]:
# MDA_MB_453_data Thi is the cell lines with proteins and the cell viability
df_drug_tgts=pd.read_csv(r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Experiments\raw_data\TNBC_set\ptv1_unique_drug_names.csv')
df_drug_tgts['drug_name_clean']=df_drug_tgts['original_drug_names']
df_drug_tgts.drop(columns=['original_drug_names'],inplace=True)

# Merge the drug_names_targets DataFrame with the MDA_MB_453_data DataFrame on the 'drug_name_clean' column
MDA_MB_453_data = pd.merge(MDA_MB_453_data, df_drug_tgts, on='drug_name_clean', how='left')
new=MDA_MB_453_data.drop(columns=['targeted_uniprots','corrected_drug_name','drugbank_targets_manual_check'])

all_targets=new['target_uniprot_ids'].str.split(',').explode().str.strip()
all_targets=set(all_targets)

overlapping_prots=set(all_targets).intersection(set(MDA_MB_453_data.columns))



In [46]:
#cleaning up import code to make the clean file overall
#files to take in:
import pandas as pd
drug_targets_path=r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Experiments\raw_data\TNBC_set\ptv1_unique_drug_names.csv'
cell_line_prots_path=r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Experiments\raw_data\TNBC_set\MDA-MB-453_processed_data.tsv'
cell_viabilities_path=r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Experiments\raw_data\TNBC_set\cell_viability_sampleinfo_for_chris.csv'

#load in the data
drug_targets=pd.read_csv(drug_targets_path)
cell_line_prots=pd.read_csv(cell_line_prots_path,sep='\t')
cell_viabilities_raw=pd.read_csv(cell_viabilities_path)

#clean up the viabilities; dropping all rows that have 
cell_1=cell_viabilities_raw.iloc[:,:4]
cell_2=cell_viabilities_raw[['Pert_iname','Inhi_5','BioRep']]
cell_viabilities=pd.concat([cell_1, cell_2], axis=1)
#dropping all the data we dont need
cell_viabilities.drop(cell_viabilities.index[cell_viabilities['Cell'] != 'MDA-MB-453'],inplace=True)
cell_viabilities.drop(cell_viabilities.index[cell_viabilities['BioRep']!=1], inplace=True)
cell_viabilities.drop(cell_viabilities.index[cell_viabilities['pert_time']!=24], inplace=True)
cell_viabilities['Pert_ID']=[phrase.split('#')[1] for phrase in cell_viabilities['Pert_ID']]
#this gets us a clean set of cell viabilities

#cleaning up cell_line_prots
clp1=cell_line_prots.filter(like='_24',axis=1)
clp2=cell_line_prots['uniprotID']
cell_line_prots=pd.concat([clp1,clp2],axis=1)
cell_line_prots.columns= [phrase[:-3] for phrase in cell_line_prots.columns]
cell_line_prots=cell_line_prots.transpose()
cell_line_prots['Drug_name']=cell_line_prots.index
cell_line_prots=cell_line_prots.reindex()
cell_line_prots.columns = cell_line_prots.loc['unipro'].values
cell_line_prots = cell_line_prots[cell_line_prots.index != 'unipro']
cell_line_prots['Pert_ID']=[phrase.split('_')[1] for phrase in cell_line_prots['Drug_name']]










KeyError: 'Drug_name'